При использовании корпуса RuTweetCorp, просьба ссылаться на следующую работу:
Рубцова Ю. Автоматическое построение и анализ корпуса коротких текстов (постов микроблогов) для задачи разработки и тренировки тонового классификатора //Инженерия знаний и технологии семантического веба. – 2012. – Т. 1. – С. 109-116.
https://study.mokoron.com/
https://elibrary.ru/item.asp?id=20399632

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier 
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

import os
import pandas as pd
import numpy as np
import time
import pickle

In [2]:
PATH_TO_DATA = '/home/atl/Yandex.Disk/temp/bases/review твиты'

In [3]:
neg, data, pos = os.listdir(PATH_TO_DATA)

In [4]:
pos_ = os.path.join(PATH_TO_DATA, pos)
neg_ = os.path.join(PATH_TO_DATA, neg)

In [5]:
pos = pd.read_csv(pos_, sep=';', names=['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', '', 'listcount'])
pos.head(2)

,id,tdate,tmane,ttext,ttype,trep,tfav,tstcount,tfol,tfrien,,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2


In [6]:
neg = pd.read_csv(neg_, sep=';', names=['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', '', 'listcount'])
neg.head(2)

,id,tdate,tmane,ttext,ttype,trep,tfav,tstcount,tfol,tfrien,,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0


In [76]:
data = pd.concat([pos[['ttext', 'ttype']], neg[['ttext', 'ttype']]], axis=0)
data.loc[:, 'ttype'] = data.loc[:, 'ttype'].map({-1: 0, 1:1}).astype(int)
print(data['ttype'].value_counts())
data.head(3)

1    114911
0    111923
Name: ttype, dtype: int64


,ttext,ttype
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1


In [77]:
data['ttype'].value_counts()

1    114911
0    111923
Name: ttype, dtype: int64

In [78]:
filename = os.path.join(PATH_TO_DATA, 'data_to_c4w4.csv')
data.to_csv(filename)

In [79]:
train, test = train_test_split(data, test_size=0.3, shuffle=True)
train['ttype'].value_counts()

1    80460
0    78323
Name: ttype, dtype: int64

In [80]:
ests = {'lr': LogisticRegression(), 'svm': LinearSVC(), 
        'sgd': SGDClassifier(), 'nb': ComplementNB(),
       'ridge': RidgeClassifier(),}

vects = {'cnt': CountVectorizer(), 'tf': TfidfVectorizer(), 'hash': HashingVectorizer()}

In [81]:
def text_classifier(estimator, vectorizer):
    return Pipeline([('vect', vectorizer), ('est', estimator)])

In [83]:
%%time

res = {}

for vect in vects:
    for est in ests:
        
        if vect == 'hash' and est == 'nb':
            continue
          
        score = round(cross_val_score(text_classifier(ests[est], vects[vect]), train.iloc[:, 0], train.iloc[:, 1], n_jobs=-1).mean(), 3)  
        res[f'{est} {vect}'] = score       

CPU times: user 9.87 s, sys: 1.85 s, total: 11.7 s
Wall time: 8min 48s


In [91]:
print(dict(sorted(res.items(), reverse=True, key=lambda x: x[1]))  )

{'lr cnt': 0.753, 'ridge tf': 0.751, 'svm hash': 0.751, 'ridge hash': 0.751, 'lr tf': 0.749, 'svm tf': 0.749, 'nb cnt': 0.745, 'sgd cnt': 0.742, 'nb tf': 0.742, 'lr hash': 0.741, 'ridge cnt': 0.739, 'svm cnt': 0.734, 'sgd tf': 0.717, 'sgd hash': 0.71}


В flask-демке указаны следующие названия для:
- модели - DefaultLogisticBigramUnprocessedTextSentiment
- vectorizer - BigramUnprocessedVectorizer
Попробуем логистическую регрессию с параметрами по умолчанию и векторайзер с биграммами.

In [84]:
%%time
vect = CountVectorizer(ngram_range=(2, 2))
lr = LogisticRegression(random_state=32, solver='lbfgs')
score = round(cross_val_score(text_classifier(lr, vect), train.iloc[:, 0], train.iloc[:, 1], n_jobs=-1).mean(), 3)  
print(f'{score} - оценка логистической регреесии с параметрами по умолчанию и CountVectorizer c биграммами')

0.698 - оценка логистической регреесии с параметрами по умолчанию и CountVectorizer c биграммами
CPU times: user 792 ms, sys: 152 ms, total: 944 ms
Wall time: 2min 22s


Как мы видим, оценка существенно разница с изначальными параметрами векторайзора. Будем для обучения использовать параметры по-умолчанию.

In [92]:
%%time
vect = CountVectorizer(ngram_range=(1, 1))
model = LogisticRegression(random_state=32, solver='lbfgs')
score = round(cross_val_score(text_classifier(lr, vect), train.iloc[:, 0], train.iloc[:, 1], n_jobs=-1).mean(), 3)
print(f'{score} - оценка логистической регреесии и CountVectorizer с параметрами по умолчанию')

0.753 - оценка логистической регреесии и CountVectorizer с параметрами по умолчанию
CPU times: user 585 ms, sys: 155 ms, total: 740 ms
Wall time: 33.1 s


In [93]:
%%time
vect.fit(train['ttext'])
X_train = vect.transform(train['ttext'])

CPU times: user 7.95 s, sys: 72.2 ms, total: 8.02 s
Wall time: 8.03 s


In [94]:
with open('UnprocessedVectorizer.pkl', 'wb') as file:
    pickle.dump(vect, file)    

In [95]:
%%time
model.fit(X_train, train['ttype'])

CPU times: user 26.3 s, sys: 13.4 ms, total: 26.3 s
Wall time: 14 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=32, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [96]:
with open('DefaultLogisticUnprocessedTextSentiment.pkl', 'wb') as file:
    pickle.dump(model, file)

Проверим работоспособность модели на тестовой выборке

In [99]:
with open('DefaultLogisticUnprocessedTextSentiment.pkl', 'rb') as file:
    model = pickle.load(file)
    
with open('UnprocessedVectorizer.pkl', 'rb') as file:
    vect = pickle.load(file)  

preds = []
trues = []

for i in range(100):
    
    text = test.iloc[i, 0]
    tp = test.iloc[i, 1]

    X_t = vect.transform([text])
    pred = model.predict(X_t)
    preds.append(pred[0])
    trues.append(tp)

results = list(map(lambda x, y: x == y, preds, trues))
f'accuracy {sum(results)}'

'accuracy 79'

In [102]:
for i in range(100):
    
    text = test.iloc[i, 0]
    tp = test.iloc[i, 1]

    X_t = vect.transform([text])
    pred = model.predict(X_t)

    
    print('Истина', tp, 'предсказание', pred[0])
    print(text)
    print()

Истина 0 предсказание 0
Хард ворк теперь:(
Гребаная сессия, закройся ка

Истина 0 предсказание 0
все меня в чём-то обвиняют разве я такая плохая?(

Истина 1 предсказание 1
моя ставка на платье на аукро пока что самая большая)уиии)оно будет мое)

Истина 0 предсказание 1
@MashaMinina3 пей вино каторое пьет мой дедуля;(

Истина 1 предсказание 1
#HappyBirthdayTreCool С Др,
Оставайся таким же заипатым чуваком ;)

Истина 0 предсказание 0
@hazza_banana69 какие проблемы ? Что случилось ?!! :(

Истина 0 предсказание 0
@followmeuou Я неттт( честно(((((( сама хотела. т.т

Истина 0 предсказание 0
И как я встану сегодня утром на Тренеровку:(

Истина 1 предсказание 1
Офигенный день!
день позитива)
бегал как идиот целый день!
танцы офигенны, хоть я и ракал но мне очень понравилось!

Истина 0 предсказание 0
@Dantelian концовка слита( а так вообще клас

Истина 0 предсказание 1
@like_a_koluchka У них какие то лица страшные(

Истина 0 предсказание 0
RT @Podslyshano: Сплю на трёх подушках и укрываюсь двум

In [105]:
text = 'ох как же интересно познавать!'
X_t = vect.transform([text])
pred = model.predict(X_t)
pred

array([1])